# Загрузка Pandas и очистка данных

In [53]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [54]:
df = pd.read_csv('main_task.csv')

In [55]:
pattern = re.compile('(\w+[\s*\w*|\s*\&*]*)')  # паттерн для типов кухонь
pattern2 = re.compile('(\d\d\/\d\d\/\d\d\d\d)')  # паттерн для дат отзывов

Ниже код для ответов на вопросы в проекте по кухням

In [56]:
cuisines = set()  # создаём пустое множество для хранения уникальных значений ингредиентов
cuisines_qty = {}  # создаём пустой словарь для хранения информации об кухнях

for cuisine_st in df['Cuisine Style']:  # начинаем перебор всех кухонь в DF
    if isinstance(cuisine_st, str):
        c = pattern.findall(cuisine_st)
        for cuisine in c:  # начинаем перебор всех кухонь в одном ресторане
            # добавляем название типа ингредиентов к множеству
            cuisines.add(cuisine)
            # проверяем есть ли кухня в словаре
            if cuisines_qty.get(cuisine) == None:
                # добавляем в словарь ключ, соответствующий очередной кухне
                cuisines_qty[cuisine] = 1
            else:
                # добавляем название типа кухни к множеству
                cuisines_qty[cuisine] += 1

len(cuisines)

# сортируем по количеству кухонь
for k in sorted(cuisines_qty, key=cuisines_qty.get, reverse=True):
    k, cuisines_qty[k]

rest_cuis_qty = {}  # создаем пустой словарь для пересчета кухонь в ресторане
# начинаем перебор всех кухонь в DF
for i, cuisine_st in enumerate(df['Cuisine Style']):
    if isinstance(cuisine_st, str):
        c = pattern.findall(cuisine_st)
        rest_cuis_qty[i] = len(c)  # количество кухонь в конкретном ресторане
    else:
        rest_cuis_qty[i] = 1  # количество кухонь в ресторане без данных
sum(rest_cuis_qty.values())/len(rest_cuis_qty)

2.6224

Код для дат отзывов, для вопросов проекта

In [57]:
dates = set()  # сет для хранения дат отзывов
differences = set()  # для хранения разностей между несколькими отзывами

for reviews in df['Reviews']:
    c = pattern2.findall(reviews)
    for date_st in c:
        datetime_object = datetime.strptime(date_st, '%m/%d/%Y')
        dates.add(datetime_object.date())
        if len(c) == 2:
            difference = datetime.strptime(
                c[0], '%m/%d/%Y') - datetime.strptime(c[1], '%m/%d/%Y')
            differences.add(abs(difference))

sorted(dates)[-1]
sorted(differences)[-20:-1]


def diff_review(review_str):
    c = pattern2.findall(review_str)
    for date_st in c:
        datetime_object = datetime.strptime(date_st, '%m/%d/%Y')
        if len(c) == 2:
            if datetime.strptime(c[0], '%m/%d/%Y') > datetime.strptime(c[1], '%m/%d/%Y'):
                return (datetime.today() - datetime.strptime(c[0], '%m/%d/%Y')).days
            else:
                return (datetime.today() - datetime.strptime(c[1], '%m/%d/%Y')).days
        elif len(c) == 1:
            return (datetime.today() - datetime.strptime(date_st, '%m/%d/%Y')).days
        else:
            return np.isnan()


df['diff_review'] = df['Reviews'].apply(diff_review)

In [58]:
def qty_cuisine(cell):
    if isinstance(cell, str):
        c = pattern.findall(cell)
        return len(c)
    else:
        return 1

#df['qty_cuisine'] = df['Cuisine Style'].apply(qty_cuisine)

In [59]:
cities = set()

for city in df['City']:
    cities.add(city)


def type_of_check(check_str):
    if check_str == '$':
        return 'низкий чек'
    elif check_str == '$$ - $$$':
        return 'средний чек'
    elif check_str == '$$$$':
        return 'высокий чек'
    else:
        return 'средний чек'


df['type_check'] = df['Price Range'].apply(type_of_check)

types_check = {'низкий чек', 'средний чек', 'высокий чек'}

In [60]:
df.diff_review.value_counts(ascending=True)


def find_item_str(cell):
    if isinstance(cell, str):
        c = pattern.findall(cell)
        if item in c:
            return 1
        return 0
    else:
        return 0


def find_item(cell):
    if item in cell:
        return 1
    return 0


# for item in cuisines:
#    df[item] = df['Cuisine Style'].apply(find_item_str)

# for item in cities:
#    df[item] = df['City'].apply(find_item)

# for item in types_check:
#    df[item] = df['type_check'].apply(find_item)

In [61]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [62]:
min_diff = df.diff_review.min()

In [63]:
df = df.drop(df.columns[[1, 2, 5, 7, 8, 9, 11]], axis=1)
df['Number of Reviews'] = df['Number of Reviews'].fillna(
    df['Number of Reviews'].median())
df['diff_review'] = df['diff_review'].fillna(df['diff_review'].mean())


def type_of_rew(cell):
    if cell < min_diff+365: #отзывы от последнего + год
        return 1
    elif cell < min_diff+365*3: # + еще 2 года
        return 2
    else:
        return 3 # максимально старые отзывы


df['type_rew'] = df['diff_review'].apply(type_of_rew)
df = df.drop(df.columns[[4]], axis=1)
df

,Restaurant_id,Ranking,Rating,Number of Reviews,type_rew
0,id_5569,5570.0,3.5,194.0,1
1,id_1535,1537.0,4.0,10.0,1
2,id_352,353.0,4.5,688.0,1
3,id_3456,3458.0,5.0,3.0,1
4,id_615,621.0,4.0,84.0,1
...,...,...,...,...,...
39995,id_499,500.0,4.5,79.0,1
39996,id_6340,6341.0,3.5,542.0,1
39997,id_1649,1652.0,4.5,4.0,2
39998,id_640,641.0,4.0,70.0,1


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [64]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis=1)
y = df['Rating']

In [65]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [66]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [67]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics  # инструменты для оценки точности модели

In [68]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [69]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.4205319279761905
